# 📦 Instalação de Dependências

In [ ]:
!pip install duckdb pandas -q

---

## ⚙️ Bloco 1: Configurações de Performance

Otimizando memória, threads e armazenamento temporário.

In [ ]:
import duckdb

con = duckdb.connect()

# Configurar para máxima performance
con.execute("SET memory_limit='2GB'")
con.execute("SET temp_directory='./temp'")
con.execute("SET threads=4")
con.execute("SET preserve_insertion_order=false")

# Verificar configurações
result = con.sql("""
    SELECT
        current_setting('memory_limit') as memoria,
        current_setting('threads') as threads,
        current_setting('temp_directory') as temp_dir
""").fetchall()

print("Configurações aplicadas:")
print(f"Memória: {result[0][0]}")
print(f"Threads: {result[0][1]}")
print(f"Temp dir: {result[0][2]}")
print("\n✅ Performance otimizada!")

---

## 🚀 Bloco 2: Prepared Statements

Reutilizando queries para melhor performance.

In [ ]:
import duckdb

con = duckdb.connect()

# Criar tabela de teste
con.execute("CREATE TABLE users (id INTEGER, name VARCHAR)")

# ✅ MELHOR: Prepared statement
stmt = con.prepare("INSERT INTO users VALUES ($1, $2)")

for i in range(100):
    stmt.execute([i, f"User_{i}"])

print(f"✅ Inseridos 100 registros com prepared statement")

# Verificar
result = con.sql("SELECT COUNT(*) FROM users").fetchone()
print(f"Total de usuários: {result[0]}")

---

## 📦 Bloco 3: Formato de Arquivo (Parquet vs CSV)

Parquet é muito mais eficiente que CSV.

In [ ]:
import duckdb
import pandas as pd
import time

con = duckdb.connect()

# Criar dados de teste
dados = pd.DataFrame({
    'id': range(10000),
    'valor': range(10000),
    'texto': [f"Item_{i}" for i in range(10000)]
})

# Salvar como CSV
dados.to_csv('test.csv', index=False)

# Salvar como Parquet
con.sql("COPY (SELECT * FROM dados) TO 'test.parquet' (FORMAT parquet)")

# Comparar velocidade de leitura
start = time.time()
csv_result = con.sql("SELECT COUNT(*) FROM 'test.csv'").fetchone()
csv_time = time.time() - start

start = time.time()
parquet_result = con.sql("SELECT COUNT(*) FROM 'test.parquet'").fetchone()
parquet_time = time.time() - start

print(f"📄 CSV: {csv_time:.4f}s")
print(f"📦 Parquet: {parquet_time:.4f}s")
print(f"⚡ Parquet é {csv_time/parquet_time:.1f}x mais rápido!")

---

## 💡 Bloco 4: Melhores Práticas

Resumo de boas práticas para performance ideal.

In [ ]:
"""
✅ MELHORES PRÁTICAS DUCKDB:

1. **Formato de Arquivo**
   - ✅ Use Parquet em vez de CSV
   - ✅ Ative compressão (ZSTD, GZIP)
   - ✅ Configure row_group_size adequadamente

2. **Conexões**
   - ✅ Reutilize conexões quando possível
   - ✅ Use context managers (with)
   - ✅ Uma conexão por thread em ambientes multi-thread

3. **Queries**
   - ✅ Use prepared statements para queries repetidas
   - ✅ Filtre dados o mais cedo possível (WHERE)
   - ✅ Limite resultados quando apropriado

4. **Memória**
   - ✅ Configure memory_limit apropriadamente
   - ✅ Use temp_directory em disco rápido
   - ✅ Habilite streaming para datasets grandes

5. **Integração Python**
   - ✅ Consulte DataFrames diretamente (sem copiar)
   - ✅ Use .df() para resultados Pandas
   - ✅ Use .arrow() para zero-copy com Arrow

6. **Compressão**
   - ✅ Databases persistentes são sempre comprimidos
   - ✅ In-memory: configure compress=True se necessário
"""

print(__doc__)
print("\n✅ Guia de melhores práticas completo!")

# Capitulo 10 Performance Boas Praticas

Notebook gerado automaticamente a partir do código fonte python.
